<font size="6">Calculate GHG Intensity per GDP, PPP </font>

In [1]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib

<font size="4">Load Environment Variables</font>

In [2]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [3]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

trino_bucket = osc.attach_s3_bucket("S3_DEV")

In [4]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

Open a Trino connection using JWT for authentication

In [5]:
# Show available schemas to ensure trino connection is set correctly
ingest_catalog = 'osc_datacommons_dev'
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('default',)
('demo_dv',)
('iceberg_demo',)
('information_schema',)
('ingest',)
('mdt_sandbox',)
('pcaf_sovereign_footprint',)
('sandbox',)


In [6]:
# define source and destination tables
# LULUCF (Land Use, Land-Use Change and Forestry)

ingest_schema = 'pcaf_sovereign_footprint'
ingest_table = 'sf_unfccc_results'
src_table_1 = 'sf_unfccc_with_lulucf'
src_table_2 = 'sf_unfccc_without_lulucf'
src_table_3 = 'sf_wdi_gdp'


In [7]:
def requantify_df(df):
    units_col = None
    columns_reversed = reversed(df.columns)
    for col in columns_reversed:
        if col.endswith("_units"):
            if units_col:
                # We expect _units column to follow a non-units column
                raise ValueError
            units_col = col
            continue
        if units_col:
            if col + '_units' != units_col:
                raise ValueError
            if (df[units_col]==df[units_col][0]).all():
                # Make a PintArray
                new_col = PintArray(df[col], dtype=f"pint[{ureg(df[units_col][0]).u}]")
            else:
                # Make a pd.Series of Quantity in a way that does not throw UnitStrippedWarning
                new_col = pd.Series(data=df[col], name=col) * pd.Series(data=df[units_col].map(lambda x: ureg(x).u), name=col)
            df = df.drop(columns=units_col)
            df[col] = new_col
            units_col = None
    return df


In [8]:
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plot
import pandas as pd
import pint
from pint import set_application_registry, Quantity
from pint_pandas import PintArray, PintType
from pint_pandas.pint_array import is_pint_type
from openscm_units import unit_registry
#PintType.ureg = unit_registry
ureg = unit_registry
Q_ = ureg.Quantity
ureg.define("CO2e=CO2=CO2eq=CO2_eq")
ureg.define("USD=[currency]=$")
set_application_registry(ureg)



sql=f"""
select country_iso_code,country_name,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_1} """ 
df1 = pd.read_sql(sql, engine)
#df1["units"] = "kt"
df1 = requantify_df(df1).convert_dtypes()
df1
df1.info(verbose=True)
sql=f"""
select country_iso_code,country_name,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_2}"""
df2 = pd.read_sql(sql, engine)
df2 = requantify_df(df2).convert_dtypes()
#df2
sql=f"""
select country_iso_code,validity_date,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_3} """
df3 = pd.read_sql(sql, engine)
df3 = requantify_df(df3).convert_dtypes()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2212 entries, 0 to 2211
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   country_iso_code  2212 non-null   string         
 1   country_name      2212 non-null   string         
 2   validity_date     2212 non-null   Int64          
 3   attribute         2212 non-null   string         
 4   value             2212 non-null   pint[CO2e * kt]
dtypes: Int64(1), pint[CO2e * kt](1), string(3)
memory usage: 88.7 KB


Read the source tables into dataframes  

<font size="4">Merge the source dataframes in a single dataframe and calculate ghg intensity values</font>

In [9]:

df_result = pd.merge(df1,df2,on=['country_iso_code','validity_date'])  
df_result=df_result.convert_dtypes()
df_result.drop(columns=['country_name_y','attribute_x','attribute_y'],inplace=True)
df_result.rename(columns={"country_name_x":"country_name","value_x": "ghg_total_with_lulucf", "value_y": "ghg_total_without_lulucf"},inplace=True)

###

df_result = pd.merge(df_result,df3,on=['country_iso_code','validity_date'],how="inner")  
df_result.rename(columns={"value":"gdp"},inplace=True)
df_result["ghg_intensity_with_lulucf_per_gdp"] = df_result["ghg_total_with_lulucf"]/df_result["gdp"]*1000000
df_result["ghg_intensity_without_lulucf_per_gdp"] = df_result["ghg_total_without_lulucf"]/df_result["gdp"]*1000000
df_result.info(verbose=True)



<class 'pandas.core.frame.DataFrame'>
Int64Index: 4185 entries, 0 to 4184
Data columns (total 8 columns):
 #   Column                                Non-Null Count  Dtype                
---  ------                                --------------  -----                
 0   country_iso_code                      4185 non-null   string               
 1   country_name                          4185 non-null   string               
 2   validity_date                         4185 non-null   Int64                
 3   ghg_total_with_lulucf                 4185 non-null   pint[CO2e * kt]      
 4   ghg_total_without_lulucf              4185 non-null   pint[CO2e * kt]      
 5   gdp                                   4185 non-null   pint[USD]            
 6   ghg_intensity_with_lulucf_per_gdp     4185 non-null   pint[CO2e * kt / USD]
 7   ghg_intensity_without_lulucf_per_gdp  4185 non-null   pint[CO2e * kt / USD]
dtypes: Int64(1), pint[CO2e * kt / USD](2), pint[CO2e * kt](2), pint[USD](1), string

In [10]:
# If DF_COL contains Pint quantities (because it is a PintArray or an array of Pint Quantities),
# return a two-column dataframe of magnitudes and units.
# If DF_COL contains no Pint quanities, return it unchanged.

def dequantify_column(df_col: pd.Series):
    if type(df_col.values)==PintArray:
        return pd.DataFrame({df_col.name: df_col.values.quantity.m,
                             df_col.name + "_units": str(df_col.values.dtype.units)},
                            index=df_col.index)
    elif df_col.size==0:
        return df_col
    elif isinstance(df_col.iloc[0], Quantity):
        values = df_col.map(lambda x: (x.m, x.u))
        return pd.DataFrame({df_col.name: df_col.map(lambda x: x.m),
                             df_col.name + "_units": df_col.map(lambda x: str(x.u))},
                            index=df_col.index)
    else:
        return df_col

# Rewrite dataframe DF so that columns containing Pint quantities are represented by a column for the Magnitude and column for the Units.
# The magnitude column retains the original column name and the units column is renamed with a _units suffix.
def dequantify_df(df):
    return pd.concat([dequantify_column(df[col]) for col in df.columns], axis=1)

In [11]:
df1 = dequantify_df(df1)
df2 = dequantify_df(df2)
df3 = dequantify_df(df3)
df_result = dequantify_df(df_result)
df_result=df_result.convert_dtypes()
df_result.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4185 entries, 0 to 4184
Data columns (total 13 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   country_iso_code                            4185 non-null   string 
 1   country_name                                4185 non-null   string 
 2   validity_date                               4185 non-null   Int64  
 3   ghg_total_with_lulucf                       4185 non-null   Float64
 4   ghg_total_with_lulucf_units                 4185 non-null   string 
 5   ghg_total_without_lulucf                    4185 non-null   Float64
 6   ghg_total_without_lulucf_units              4185 non-null   string 
 7   gdp                                         4185 non-null   Float64
 8   gdp_units                                   4185 non-null   string 
 9   ghg_intensity_with_lulucf_per_gdp           4185 non-null   Float64
 10  ghg_intensit

In [12]:

# make sure schema exists, or table creation below will fail in weird ways
sql = f"""
CREATE SCHEMA if not exists {ingest_catalog}.{ingest_schema}
 AUTHORIZATION USER mersin35
 WITH (
     location = 's3a://osc-datacommons-s3-bucket-dev02/data/pcaf_covereign.db'
 )
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())


CREATE SCHEMA if not exists osc_datacommons_dev.pcaf_sovereign_footprint
 AUTHORIZATION USER mersin35
 WITH (
     location = 's3a://osc-datacommons-s3-bucket-dev02/data/pcaf_covereign.db'
 )

[(True,)]


<font size="5">Save the results in Trino</font>


In [13]:

# make sure schema exists, or table creation below will fail in weird ways
sql = f"""
CREATE SCHEMA if not exists {ingest_catalog}.{ingest_schema}
 AUTHORIZATION USER mersin35
 WITH (
     location = 's3a://osc-datacommons-s3-bucket-dev02/data/pcaf_covereign.db'
 )
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())


CREATE SCHEMA if not exists osc_datacommons_dev.pcaf_sovereign_footprint
 AUTHORIZATION USER mersin35
 WITH (
     location = 's3a://osc-datacommons-s3-bucket-dev02/data/pcaf_covereign.db'
 )

[(True,)]


In [14]:
import osc_ingest_trino as osc
#df_result['validity_date']=pd.to_datetime(df_result['validity_date'], unit='D')
#df_result=df_result.convert_dtypes()
#df_result.info()
columnschema = osc.create_table_schema_pairs(df_result) 
sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())




drop table if exists osc_datacommons_dev.pcaf_sovereign_footprint.sf_unfccc_results

[(True,)]


In [15]:
tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['country_iso_code']
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())


create table if not exists osc_datacommons_dev.pcaf_sovereign_footprint.sf_unfccc_results(
    country_iso_code varchar,
    country_name varchar,
    validity_date bigint,
    ghg_total_with_lulucf double,
    ghg_total_with_lulucf_units varchar,
    ghg_total_without_lulucf double,
    ghg_total_without_lulucf_units varchar,
    gdp double,
    gdp_units varchar,
    ghg_intensity_with_lulucf_per_gdp double,
    ghg_intensity_with_lulucf_per_gdp_units varchar,
    ghg_intensity_without_lulucf_per_gdp double,
    ghg_intensity_without_lulucf_per_gdp_units varchar
) with (
    format = 'ORC',
    partitioning = array['country_iso_code']
)

[(True,)]


In [16]:
# Delete all data from our db, so we start with empty table
sql=f"""
delete from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
qres = engine.execute(sql)
print(qres.fetchall())

[(None,)]


In [17]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
pd.read_sql(sql, engine)


,country_iso_code,country_name,validity_date,ghg_total_with_lulucf,ghg_total_with_lulucf_units,ghg_total_without_lulucf,ghg_total_without_lulucf_units,gdp,gdp_units,ghg_intensity_with_lulucf_per_gdp,ghg_intensity_with_lulucf_per_gdp_units,ghg_intensity_without_lulucf_per_gdp,ghg_intensity_without_lulucf_per_gdp_units


In [18]:
print(ingest_catalog)
#df=df.drop(df[df.country_name=="cote d'ivoire"].index)
df_result.to_sql(ingest_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 1000, verbose = True))

osc_datacommons_dev
constructed fully qualified table name as: "pcaf_sovereign_footprint.sf_unfccc_results"
inserting 1000 records
  ('TUV', 'Tuvalu', 2014, 18.40074, 'CO2e * kt', 18.43414, 'CO2e * kt', 37290607.5356048, 'USD', 0.49344167917969983, 'CO2e * kt / USD', 0.4943373470759149, 'CO2e * kt / USD')
  ('TUV', 'Tuvalu', 2014, 18.40074, 'CO2e * kt', 18.43414, 'CO2e * kt', 36402184.4892923, 'USD', 0.5054844993001059, 'CO2e * kt / USD', 0.5064020266537136, 'CO2e * kt / USD')
  ('SGP', 'Singapore', 1994, 26859.08, 'CO2e * kt', 26859.08, 'CO2e * kt', 113054534233.087, 'USD', 0.23757631820961775, 'CO2e * kt / USD', 0.23757631820961775, 'CO2e * kt / USD')
  ...
  ('DOM', 'Dominican Republic', 1990, 7084.93, 'CO2e * kt', 12640.92, 'CO2e * kt', 7073675544.80847, 'USD', 1.0015910335610163, 'CO2e * kt / USD', 1.7870370106637783, 'CO2e * kt / USD')
batch insert result: [(1000,)]
inserting 1000 records
  ('DOM', 'Dominican Republic', 1990, 7084.93, 'CO2e * kt', 12640.92, 'CO2e * kt', 261516834

In [19]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}"""
pd.read_sql(sql, engine)


,country_iso_code,country_name,validity_date,ghg_total_with_lulucf,ghg_total_with_lulucf_units,ghg_total_without_lulucf,ghg_total_without_lulucf_units,gdp,gdp_units,ghg_intensity_with_lulucf_per_gdp,ghg_intensity_with_lulucf_per_gdp_units,ghg_intensity_without_lulucf_per_gdp,ghg_intensity_without_lulucf_per_gdp_units
0,CMR,Cameroon,1994,187911.390000,CO2e * kt,165725.023000,CO2e * kt,2.226333e+10,USD,8.440400,CO2e * kt / USD,7.443856,CO2e * kt / USD
1,CMR,Cameroon,1994,187911.390000,CO2e * kt,165725.023000,CO2e * kt,8.902446e+09,USD,21.107838,CO2e * kt / USD,18.615673,CO2e * kt / USD
2,CMR,Cameroon,2000,-47642.000000,CO2e * kt,28939.000000,CO2e * kt,3.147509e+10,USD,-1.513642,CO2e * kt / USD,0.919426,CO2e * kt / USD
3,CMR,Cameroon,2000,-47642.000000,CO2e * kt,28939.000000,CO2e * kt,1.056658e+10,USD,-4.508744,CO2e * kt / USD,2.738729,CO2e * kt / USD
4,BGR,Bulgaria,2014,48245.390000,CO2e * kt,57987.867349,CO2e * kt,5.708201e+10,USD,0.845194,CO2e * kt / USD,1.015869,CO2e * kt / USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4180,PRY,Paraguay,2005,107347.829000,CO2e * kt,32296.219000,CO2e * kt,4.359509e+10,USD,2.462383,CO2e * kt / USD,0.740822,CO2e * kt / USD
4181,PRY,Paraguay,2012,167378.346300,CO2e * kt,45230.726300,CO2e * kt,6.433533e+10,USD,2.601655,CO2e * kt / USD,0.703046,CO2e * kt / USD
4182,PRY,Paraguay,2015,51025.840000,CO2e * kt,35356.690000,CO2e * kt,7.683044e+10,USD,0.664136,CO2e * kt / USD,0.460191,CO2e * kt / USD
4183,PRY,Paraguay,2015,51025.840000,CO2e * kt,35356.690000,CO2e * kt,3.621137e+10,USD,1.409111,CO2e * kt / USD,0.976397,CO2e * kt / USD
